# Webscraping

## Goodfeeds
Goodfeeds es una página que recoge todos los feeds que se publican en Bluesky. Tiene una sección de los 'Top 50', esta es la que se va a utilizar a usar para el análisis.

https://goodfeeds.co/all

In [107]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [73]:
url_principal = "https://goodfeeds.co/all?p=1"

In [75]:
response = requests.get(url_principal)
soup = BeautifulSoup(response.text, "html.parser")
print(response)
print(bool(response))

<Response [200]>
True


In [182]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>All — goodfeeds</title>
<meta content="All — goodfeeds" property="og:title">
<meta content="Custom feeds on Bluesky" property="og:description">
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css2?family=Quicksand:wght@300;600&amp;display=swap" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" rel="stylesheet"/>
<script crossorigin="anonymous" integrity="sha384-geWF76RCwLtnZ8qwWowPQNguL3RmwHVBC9FhGdlKrxdiJJigb/j/68SIy3Te4Bkz" src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
<style type="text/css">
header.topbar {
  back

In [79]:
# Definición del objeto que contiene todos los feeds
tarjeta = soup.find_all('div', attrs = 'flex-grow-1')

In [91]:
# Prueba con 1 tarjeta
tarjeta_1 = tarjeta[0]

# feed
tarjeta_1.find_all('strong')[0].get_text(strip = True)

# user
tarjeta_1.find_all('a')[1].get_text(strip = True)

# description
tarjeta_1.find('p', attrs = {'class':'text-break'}).get_text(strip = True)

# likes
tarjeta_1.find('p', attrs = {'class':'small'}).get_text(strip = True)

'♡ 37,972 likes'

In [103]:
def feeds_page(div):
    '''Recibe un div del que sacar info y devuelve una lista de diccionarios con todos los feeds encontrados'''
    feeds_pag_1 = []

    for d in div:
        feed = {'feed':'', 'user':'', 'description':'', 'likes':''}
        feed['feed'] = d.find_all('strong')[0].get_text(strip = True)
        feed['user'] = d.find_all('a')[1].get_text(strip = True)
        feed['description'] = d.find('p', attrs = {'class':'text-break'}).get_text(strip = True)
        feed['likes'] = d.find('p', attrs = {'class':'small'}).get_text(strip = True)
        feeds_pag_1.append(feed)
    
    return feeds_pag_1


In [175]:
def div_finder(n):
    '''Busca un div en el número de página que se le pasa por argumento y lo devuelve'''
    url_principal = f"https://goodfeeds.co/all?p={n}"
    response = requests.get(url_principal)
    soup = BeautifulSoup(response.text, "html.parser")
    print('Pág.', n, response)

    div = soup.find_all('div', attrs = 'flex-grow-1')
    time.sleep(3)
    return div

In [184]:
# Lista de diccionarios con todos los feeds que se encuentren por página
pages = 20
pages_list = []

for i in range(1,pages+1):
    pages_list.append(feeds_page(div_finder(i)))

Pág. 1 <Response [200]>
Pág. 2 <Response [200]>
Pág. 3 <Response [200]>
Pág. 4 <Response [200]>
Pág. 5 <Response [200]>
Pág. 6 <Response [200]>
Pág. 7 <Response [200]>
Pág. 8 <Response [200]>
Pág. 9 <Response [200]>
Pág. 10 <Response [200]>
Pág. 11 <Response [200]>
Pág. 12 <Response [200]>
Pág. 13 <Response [200]>
Pág. 14 <Response [200]>
Pág. 15 <Response [200]>
Pág. 16 <Response [200]>
Pág. 17 <Response [200]>
Pág. 18 <Response [200]>
Pág. 19 <Response [200]>
Pág. 20 <Response [200]>


In [186]:
# Lista de dataframes con cada uno de las listas de feeds anteriores convertidas a df

df_list = []

for n in range(len(pages_list)):
    df_list.append(pd.DataFrame(pages_list[n]))


In [198]:
# Concatenación de todos los dataframes de la lista anterior en uno solo
df_bluesky_feeds = pd.concat(df_list, ignore_index=True)
df_bluesky_feeds

,feed,user,description,likes
0,Popular With Friends,@bsky.app,A mix of popular content from accounts you fol...,"♡ 37,972 likes"
1,Discover,@bsky.app,Trending content from your personal network,"♡ 36,646 likes"
2,📌,@jaz.bsky.social,A feed of posts you've replied to with a 📌.,"♡ 35,769 likes"
3,Artists: Trending,@bsky.art,General art feed — image posts from artists ac...,"♡ 31,141 likes"
4,Blacksky,@rudyfraser.com,An algorithm showcasing posts by Black users o...,"♡ 27,255 likes"
...,...,...,...,...
195,#EduSky 🍎,@afreeparticle.com,A feed for discussion related to education.\n\...,♡ 722 likes
196,Feed Paranormal,@apogrifa.tripazeada.club,CAPA POR @ivandala.bsky.social. \n\nUse as tag...,♡ 721 likes
197,Indie Devs & Games,@patrickhenschel.de,Pure #IndieDev #IndieGame #SoloDev #IndieGameD...,♡ 716 likes
198,Gaming,@sweetbee.vip,"All-inclusive, chronological video games feed,...",♡ 715 likes


In [200]:
df_bluesky_feeds['likes'] = df_bluesky_feeds['likes'].map(lambda x: x.strip('♡ '))
df_bluesky_feeds['likes'] = df_bluesky_feeds['likes'].map(lambda x: x.strip(' likes'))
df_bluesky_feeds

,feed,user,description,likes
0,Popular With Friends,@bsky.app,A mix of popular content from accounts you fol...,"37,972"
1,Discover,@bsky.app,Trending content from your personal network,"36,646"
2,📌,@jaz.bsky.social,A feed of posts you've replied to with a 📌.,"35,769"
3,Artists: Trending,@bsky.art,General art feed — image posts from artists ac...,"31,141"
4,Blacksky,@rudyfraser.com,An algorithm showcasing posts by Black users o...,"27,255"
...,...,...,...,...
195,#EduSky 🍎,@afreeparticle.com,A feed for discussion related to education.\n\...,722
196,Feed Paranormal,@apogrifa.tripazeada.club,CAPA POR @ivandala.bsky.social. \n\nUse as tag...,721
197,Indie Devs & Games,@patrickhenschel.de,Pure #IndieDev #IndieGame #SoloDev #IndieGameD...,716
198,Gaming,@sweetbee.vip,"All-inclusive, chronological video games feed,...",715


In [202]:
df_bluesky_feeds.to_csv('bsky_feeds.csv', index = False)

## Bluesky Blog
El blog de Bluesky recoge todas las noticias propias de la plataforma que han ido publicando desde su apertura en 2022. Se realiza un scraping de todos los titulares.

https://bsky.social/about/blog

In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from forecasting_bluesky_code import text_cleaning as tc

In [8]:
# URL principal
url = "https://bsky.social/about/blog"

# Response
response = requests.get(url)

# Comprobación
print(bool(response))

# Objeto soup
soup = BeautifulSoup(response.text, "html.parser")

True


In [9]:
soup

<!DOCTYPE html>
<html lang="en"><head><title>Blog - Bluesky</title><link href="/about/images/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="/about/images/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/about/images/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/about/images/site.webmanifest" rel="manifest"/><link color="#044318" href="/about/images/safari-pinned-tab.svg" rel="mask-icon"/><meta content="#044318" name="msapplication-TileColor"/><meta content="#044318" name="theme-color"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="Bluesky" property="og:site_name"/><meta content="Blog - Bluesky" property="og:title"/><meta content="https://bsky.social/about/blog" property="og:url"/><meta content="https://bsky.social/about/blog" name="twitter:url"/><meta content="https://bsky.social/about/images/social-card-default-gradient.png" property="og:image"/

In [10]:
# Se define la sección que recoge todos los posts del blog
section = soup.find('section', attrs = {'class':'max-w-[680px] px-4 600:px-6 mx-auto mt-5 800:mt-24'})
all_cards = section.find_all('a')

In [26]:
# Pruebas con la info de una tarjeta
all_cards[0].find_all('h4')[0].get_text(strip = True) # title
all_cards[0].find('p', attrs = {'class':'font-600'}).get_text(strip = True) # date
all_cards[0].find('p', attrs = {'class':'text-sans-16 -tracking-[0.16px] 800:text-sans-18 800:-tracking-[0.18px]'}).get_text(strip = True) # description


'You asked, we listened.'

In [20]:
# Se recorren todas las tarjetas para sacar la información de los artículos y guardarlos en listas
articles = []
for card in all_cards:
    article = []
    article.append(card.find_all('h4')[0].get_text(strip = True)) # title
    article.append(card.find('p', attrs = {'class':'font-600'}).get_text(strip = True)) # date
    article.append(card.find('p', attrs = {'class':'text-sans-16 -tracking-[0.16px] 800:text-sans-18 800:-tracking-[0.18px]'}).get_text(strip = True)) # description

    articles.append(article)

### Dataset

In [253]:
df_bsky_news = pd.DataFrame(articles, columns = ['title', 'date', 'description'])

In [267]:
# Se crea una columna subject = Bluesky
df_bsky_news['subject'] = 'bluesky'


In [269]:
# Se eliminan noticias en portugués y las del 1 de abril (fools day)
index_to_drop = [0,21,8, 10, 20]
df_bsky_news.drop(index_to_drop, inplace=True)

In [271]:
df_bsky_news.reset_index(inplace=True)

In [273]:
# Hay una description vacía, lo completamos manualmente
df_bsky_news.loc[23,'description'] = 'Journalists and news organizations can self-verify by setting their website directly as their username.'


In [275]:
# Como la descripción queda pobre, se junta con el titular en una única columna 'headline'
df_bsky_news['headline'] = df_bsky_news['title'] + ' ' + df_bsky_news['description']

In [279]:
df_bsky_news.columns

Index(['index', 'title', 'description', 'year', 'day', 'month', 'date',
       'subject', 'headline'],
      dtype='object')

In [281]:
# Se eliminan las columnas que sobran y se reordenan para exportar
df_bsky_news.drop(columns = (['title','description']), inplace=True)

df_bsky_news = df_bsky_news[['date','year','month','day','headline','subject']]

In [291]:
# Se eliminan las noticias de 2022 (previas a la apertura a todo el público de Bluesky)
df_bsky_news = df_bsky_news[df_bsky_news['year'] > 2022]

In [293]:
df_bsky_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 38
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      39 non-null     datetime64[ns]
 1   year      39 non-null     int64         
 2   month     39 non-null     int64         
 3   day       39 non-null     int64         
 4   headline  39 non-null     object        
 5   subject   39 non-null     object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 2.1+ KB


In [295]:
# Se exporta
df_bsky_news.to_csv('bsky_news.csv', index = False)